In [15]:

''' Rewrite the basic sliders example as a standalone document with JavaScript callbacks.
Present an interactive function explorer with slider widgets.
Scrub the sliders to change the properties of the ``sin`` curve, or
type into the title text box to update the title of the plot.
Run example with python sliders-standalone-code.py
In this version, the callbacks are written as strings of JS code.
'''
import numpy as np

from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.callbacks import CustomJS
from bokeh.models.widgets import Slider, TextInput, Button
from bokeh.plotting import figure, output_notebook, output_file, show

output_notebook()
output_file("sliders-standalone.html")
# Set up data
N = 100
x = np.zeros(N)
y = np.zeros(N)
a = 42164 # km
e = 0.7
source = ColumnDataSource(data=dict(x=x, y=y))


# Set up plot
plot = figure(plot_height=400, plot_width=400, title="my orbit",
              x_range=[-50000, 50000], y_range=[-50000, 50000], toolbar_location=None)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


# Set up widgets
text = TextInput(title="title", value='my orbit')
sma = Slider(title="semi-major axis", value=a, start=6378, end=50000, step=10)
eccentricity = Slider(title="eccentricity", value=e, start=0, end=1, step=0.01)

# Set up callbacks
text_callback_code ="""
    plot.title.text = text.value
"""

text.js_on_change('value', CustomJS(args=dict(text=text,plot=plot), code=text_callback_code))

sliders_callback_code="""
    console.log("coucou from callback")
    const a = sma.value,
        e = eccentricity.value,
        N = 100,
        b = a * window.Math.sqrt(1 - e**2),
        c = window.Math.sqrt(a**2 - b**2),
        x=source.data['x'],
        y=source.data['y']
    console.log("coucou after const")
    for(let i=0; i < N; i++) {
        x[i] = c + a * window.Math.cos(i/(N-1) * 2 * window.Math.PI)
        console.log(x[i])
        y[i] = b * window.Math.sin(i/(N-1) * 2 * window.Math.PI)
        console.log(y[i])
    }
    console.log("coucou after loop")
    source.change.emit()
"""

slider_args = dict(source=source, sma=sma, eccentricity=eccentricity)

for w in [sma, eccentricity]:
    w.js_on_change('value', CustomJS(args=slider_args, code=sliders_callback_code))

# Set up layouts and add to document
inputs = widgetbox(text, sma, eccentricity)

show(row(inputs, plot, width=800))

Loading BokehJS ...